# 🔥 Hybrid Deep Learning & Gradient Boosting for Multi-Stock Time Series Forecasting: A Kaggle Grandmaster Pipeline

**Alternative shorter version:**  
*End-to-End Multi-Stock Forecasting with XGBoost, LSTM, TFT & N-BEATS*

**SEO-friendly version:**  
*Stock Price Forecasting with XGBoost, LightGBM, CatBoost, LSTM, TFT & N-BEATS — Complete Kaggle Pipeline*

---

## 📄 Notebook Overview

### 🔎 Overview
This notebook presents a complete end-to-end framework for **multi-stock time series forecasting** using both **classical machine learning** and **state-of-the-art deep learning architectures**.

We build a production-ready forecasting pipeline covering:

- Robust preprocessing & feature engineering  
- Time-aware cross-validation (`TimeSeriesSplit`)  
- Gradient Boosting models (**XGBoost, LightGBM, CatBoost**)  
- Deep Learning models (**LSTM + GRU**)  
- Advanced architectures (**Temporal Fusion Transformer & N-BEATS**)  
- Ensemble blending strategies  
- Kaggle-ready submission generation  

The dataset includes multiple semiconductor stocks (**AMD, ASML, INTC, NVDA**), and models are trained per stock to capture individual market dynamics.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Classical
from statsmodels.tsa.arima.model import ARIMA
from prophet import Prophet

# ML
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error

import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor

# DL
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Bidirectional

# Transformers
import torch
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer, NBeats
from pytorch_forecasting.metrics import QuantileLoss
from pytorch_lightning import Trainer

## 🧾 Code Cell – Load Data

In [ ]:
import pandas as pd

# ===============================
# 1. Load CSV with Multi-Header
# ===============================
df = pd.read_csv("ai_chip_stocks_2018_2026.csv", header=[0,1,2])

# ===============================
# 2. Fix Column Names
# ===============================
df.columns = [
    f"{c2}_{c0}" if c1 == "" else f"{c1}_{c0}"
    for c0, c1, c2 in df.columns
]

# First column is Date
df.rename(columns={df.columns[0]: "Date"}, inplace=True)

# ===============================
# 3. Convert Date
# ===============================
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
df = df.dropna(subset=["Date"])

# ===============================
# 4. Reshape to Long Format
# ===============================
tickers = ["AMD", "ASML", "INTC", "NVDA"]
ohlcv = ["Open", "High", "Low", "Close", "Volume"]

data_list = []

for t in tickers:
    cols = [c for c in df.columns if c.startswith(t)]
    temp = df[["Date"] + cols].copy()
    temp.columns = ["Date"] + ohlcv
    temp["Ticker"] = t
    data_list.append(temp)

data = pd.concat(data_list, ignore_index=True)

# ===============================
# 5. Sort & Add time_idx
# ===============================
data.sort_values(["Ticker","Date"], inplace=True)
data["time_idx"] = data.groupby("Ticker").cumcount()

# ===============================
# 6. Done
# ===============================
print("✅ Data processed successfully")
display(data.head())


## 🛠 Feature Engineering

In [ ]:
def add_features(data):
    data = data.copy()
    
    # Lag features
    for lag in [1, 3, 7, 14, 30]:
        data[f"lag_{lag}"] = data.groupby("Ticker")["Close"].shift(lag)
    
    # Rolling means (shifted to avoid leakage)
    data["rolling_7"] = data.groupby("Ticker")["Close"].shift(1).rolling(7).mean()
    data["rolling_14"] = data.groupby("Ticker")["Close"].shift(1).rolling(14).mean()
    data["rolling_30"] = data.groupby("Ticker")["Close"].shift(1).rolling(30).mean()

    # Rolling volatility
    data["volatility_7"] = data.groupby("Ticker")["Close"].shift(1).rolling(7).std()
    data["volatility_14"] = data.groupby("Ticker")["Close"].shift(1).rolling(14).std()

    return data


# Apply features
data = add_features(data)

# Drop rows with NaNs from lags/rolling
data = data.dropna().reset_index(drop=True)

display(data.head())


## 🔁 TimeSeries CV

In [ ]:
# ✅ Import TimeSeriesSplit
from sklearn.model_selection import TimeSeriesSplit

# TimeSeries Cross-Validation
tscv = TimeSeriesSplit(n_splits=5)

# Select feature columns (lags + rolling + volatility)
features = [c for c in df.columns 
            if c.startswith("lag_") 
            or c.startswith("rolling_") 
            or c.startswith("volatility_")]

# Target column
target = "Close"

print("Features:", features)
print("Target:", target)

## 🌲 XGBoost / LightGBM / CatBoost

In [ ]:
# ===============================
# 0. Preprocess DataFrame columns
# ===============================
# Strip whitespace and lowercase all column names
df.columns = df.columns.str.strip().str.lower()

# ===============================
# 1. Define Models
# ===============================
models = {
    "XGBoost": xgb.XGBRegressor(n_estimators=300, learning_rate=0.05, random_state=42),
    "LightGBM": lgb.LGBMRegressor(n_estimators=300, random_state=42),
    "CatBoost": CatBoostRegressor(iterations=300, verbose=0, random_state=42)
}

# ===============================
# 2. List of Stocks
# ===============================
stocks = ["amd", "asml", "intc", "nvda"]

# ===============================
# 3. Cross-Validation for Each Stock
# ===============================
results = {}

for stock in stocks:
    print(f"\n===== Processing {stock.upper()} =====")
    
    target_col = f"{stock}_close"
    if target_col not in df.columns:
        print(f"Skipping {stock}: target column '{target_col}' not found")
        continue
    
    # Use all columns except the target and date as features
    X = df[[col for col in df.columns if col != target_col and col != "date"]]
    y = df[target_col]

    # TimeSeries CV
    tscv = TimeSeriesSplit(n_splits=5)
    cv_scores = {}
    
    for name, model in models.items():
        maes = []
        for train_idx, val_idx in tscv.split(X):
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
            
            model.fit(X_train, y_train)
            preds = model.predict(X_val)
            maes.append(mean_absolute_error(y_val, preds))
        
        cv_scores[name] = np.mean(maes)
    
    results[stock] = cv_scores

# ===============================
# 4. Display Results
# ===============================
for stock, scores in results.items():
    print(f"\n{stock.upper()} CV Results:")
    display(pd.DataFrame.from_dict(scores, orient='index', columns=['MAE']).sort_values('MAE'))


## 🤖 LSTM / GRU / BiLSTM

In [ ]:
# ===============================
# 0. Clean column names
# ===============================
df.columns = df.columns.str.strip().str.lower()

# ===============================
# 1. Select Target Stock
# ===============================
target_stock = "amd"   # 🔥 change to: "asml", "intc", "nvda"
target_col = f"{target_stock}_close"

if target_col not in df.columns:
    raise ValueError(f"{target_col} not found in DataFrame")

# ===============================
# 2. Scale Target
# ===============================
scaler = MinMaxScaler()
df[f"{target_col}_scaled"] = scaler.fit_transform(df[[target_col]])

# ===============================
# 3. Sequence Generator
# ===============================
def make_seq(series, window=30):
    X, y = [], []
    for i in range(window, len(series)):
        X.append(series[i-window:i])
        y.append(series[i])
    return np.array(X), np.array(y)

X_seq, y_seq = make_seq(df[f"{target_col}_scaled"].values, window=30)

# Reshape for RNN input: (samples, timesteps, features)
X_seq = X_seq.reshape((X_seq.shape[0], X_seq.shape[1], 1))

# ===============================
# 4. Build DL Model
# ===============================
model_dl = Sequential([
    Bidirectional(LSTM(64, return_sequences=True), input_shape=(30, 1)),
    GRU(32),
    Dense(1)
])

model_dl.compile(optimizer="adam", loss="mse")

# ===============================
# 5. Train
# ===============================
model_dl.fit(X_seq, y_seq, epochs=15, batch_size=32, verbose=1)


## 🔮 TFT & N-BEATS

In [ ]:
# ===============================
# 0. Clean columns
# ===============================
df.columns = df.columns.str.strip().str.lower()

# ===============================
# 1. Select Stock
# ===============================
target_stock = "amd"   # asml / intc / nvda
target_col = f"{target_stock}_close"

# ===============================
# 2. time_idx + group
# ===============================
df = df.sort_values("date").reset_index(drop=True)
df["time_idx"] = np.arange(len(df))
df["series_id"] = target_stock

# ===============================
# 3. TFT Dataset (with features)
# ===============================
max_encoder_length = 60
max_prediction_length = 30

tft_dataset = TimeSeriesDataSet(
    df,
    time_idx="time_idx",
    target=target_col,
    group_ids=["series_id"],
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    time_varying_known_reals=["time_idx"],
    time_varying_unknown_reals=[target_col],
)

tft_loader = tft_dataset.to_dataloader(train=True, batch_size=64, num_workers=0)

# ===============================
# 4. Train TFT
# ===============================
tft = TemporalFusionTransformer.from_dataset(
    tft_dataset,
    learning_rate=0.001,
    loss=QuantileLoss(),
    hidden_size=16,
    attention_head_size=2,
    dropout=0.1
)

trainer = Trainer(max_epochs=10, accelerator="auto")
trainer.fit(tft, train_dataloaders=tft_loader)

# ===============================
# 5. N-BEATS Dataset (TARGET ONLY!)
# ===============================
nbeats_dataset = TimeSeriesDataSet(
    df[["time_idx", "series_id", target_col]],  # 👈 ONLY target
    time_idx="time_idx",
    target=target_col,
    group_ids=["series_id"],
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    time_varying_unknown_reals=[target_col],
)

nbeats_loader = nbeats_dataset.to_dataloader(train=True, batch_size=64, num_workers=0)

# ===============================
# 6. Train N-BEATS
# ===============================
nbeats = NBeats.from_dataset(
    nbeats_dataset,
    learning_rate=0.001
)

trainer.fit(nbeats, train_dataloaders=nbeats_loader)


## 🧮 Ensemble + Leaderboard Tricks

In [ ]:
# ===============================
# 0. Clean column names
# ===============================
df.columns = df.columns.str.strip().str.lower()

# ===============================
# 1. Select Target Stock
# ===============================
target_stock = "amd"   # 🔁 asml / intc / nvda
target_col = f"{target_stock}_close"

if target_col not in df.columns:
    raise ValueError(f"{target_col} not found in DataFrame")

# ===============================
# 2. Build Feature Matrix (NUMERIC ONLY)
# ===============================
# Drop target, date, and any object/categorical columns
drop_cols = [target_col, "date", "series_id"]
features = [c for c in df.columns if c not in drop_cols]

X = df[features].select_dtypes(include=["number"]).copy()
y = df[target_col].copy()

# ===============================
# 3. Define Models
# ===============================
models = {
    "xgb": xgb.XGBRegressor(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    ),
    "lgb": lgb.LGBMRegressor(
        n_estimators=300,
        learning_rate=0.05,
        num_leaves=31,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    ),
    "cat": CatBoostRegressor(
        iterations=300,
        learning_rate=0.05,
        depth=6,
        verbose=0,
        random_state=42
    )
}

# ===============================
# 4. Fit Models on FULL Data
# ===============================
for name, model in models.items():
    model.fit(X, y)

# ===============================
# 5. Ensemble Predictions
# ===============================
pred_xgb = models["xgb"].predict(X)
pred_lgb = models["lgb"].predict(X)
pred_cat = models["cat"].predict(X)

# Average blend
final_preds = (pred_xgb + pred_lgb + pred_cat) / 3

# ===============================
# 6. Leaderboard Trick
# ===============================
final_preds = np.maximum(final_preds, 0)

print("Final predictions shape:", final_preds.shape)
